# Loading data 

In [3]:
import pandas as pd

path = r"C:\Users\Jesus Sanchez\Desktop\ALEXIS\1. Pre-Trabajo\1. Supply Chain Intelligence\data\raw\online_retail_II.xlsx"

# 1. Leer todas las hojas
df_dict = pd.read_excel(path, sheet_name=None)

# 2. Añadir una columna identificadora a cada hoja antes de unir
frames = []
for sheet_name, frame in df_dict.items():
    frame['Period'] = sheet_name  # Crea una columna con el nombre de la hoja (e.g., 'Year 2009-2010')
    frames.append(frame)

# 3. Concatenar con la nueva columna
df = pd.concat(frames, ignore_index=True)

print(f"Registros totales: {df.shape[0]}")
print(f"Columnas actuales: {df.columns.tolist()}") # Verás que ahora 'Period' existe

Registros totales: 1067371
Columnas actuales: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Period']


# Inspecting data

In [5]:
# 1. Ver tipos de datos y nulos por columna
print("--- Resumen de Columnas y Nulos ---")
print(df.info())

# 2. Ver cuántos nulos hay exactamente por columna
print("\n--- Conteo de Valores Nulos ---")
print(df.isnull().sum())

# 3. Ver una muestra de los datos para entender las columnas
print("\n--- Muestra de los datos ---")
display(df.head())

# 4. Resumen estadístico con formato por columna
# Definimos el formato: Quantity como número limpio y Price como moneda
resumen_estilo = df[["Quantity", "Price"]].describe().style.format({
    "Quantity": "{:,.2f}",
    "Price": "£{:,.2f}"
})

display(resumen_estilo)


--- Resumen de Columnas y Nulos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
 8   Period       1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 73.3+ MB
None

--- Conteo de Valores Nulos ---
Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country          

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Period
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,Year 2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Year 2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Year 2009-2010
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,Year 2009-2010
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,Year 2009-2010


,Quantity,Price
count,"1,067,371.00","£1,067,371.00"
mean,9.94,£4.65
std,172.71,£123.55
min,"-80,995.00","£-53,594.36"
25%,1.00,£1.25
50%,3.00,£2.10
75%,10.00,£4.15
max,"80,995.00","£38,970.00"


# Cleaning data

In [6]:
# 1. Eliminar filas sin Descripción
# Si no sabemos qué es el producto, no podemos predecir su demanda.
df_clean = df.dropna(subset=['Description'])

# 2. Ver el resultado final de la limpieza
print(f"Registros después de la limpieza: {len(df_clean):,}")
print(f"Registros eliminados: {len(df) - len(df_clean):,}")

Registros después de la limpieza: 1,062,989
Registros eliminados: 4,382


# Guardando base de datos en formato CSV

In [7]:
# Guardamos en la carpeta de datos procesados. 
# index=False evita que se cree una columna extra con los números de fila.
output_path = r'C:\Users\Jesus Sanchez\Desktop\ALEXIS\1. Pre-Trabajo\1. Supply Chain Intelligence\data\processed\cleaned_retail_data.csv'
df_clean.to_csv(output_path, index=False)


# Guardando base de datos en formato SQLite

In [10]:
import sqlite3

# 1. Crear conexión a la base de datos (se creará el archivo si no existe)
# Lo guardaremos en la carpeta 'data/' que creamos antes
conn = sqlite3.connect(r'C:\Users\Jesus Sanchez\Desktop\ALEXIS\1. Pre-Trabajo\1. Supply Chain Intelligence\data\processed\retail_vault.db')

# 2. Guardar el DataFrame limpio en una tabla llamada 'transactions'
print("📦 Guardando datos en SQLite... (esto será rápido)")
df_clean.to_sql('transactions', conn, if_exists='replace', index=False)

# 3. Cerrar conexión
conn.close()
print("✅ ¡Base de datos 'retail_vault.db' creada con éxito en la carpeta data/!")

📦 Guardando datos en SQLite... (esto será rápido)
✅ ¡Base de datos 'retail_vault.db' creada con éxito en la carpeta data/!
